In [2]:
import json
import os
import pandas as pd

transcript_dir = '../../data/transcripts'
annotations = json.load(open('../../data/annotation.json'))
video_lengths = {date: x[-1]['video_end'] for date, x in annotations.items()}

In [3]:
def construct(split, window_length):
    '''
        Construct the start and end times and corresponding transcripts for split `split` with w = `window_length` seconds
    '''
    window_radius = window_length / 2
    frames_fn = f'{split}.json'
    frames = json.load(open(frames_fn))

    data = {}
    for date in frames:
        print('Processing:', date)
        transcripts = pd.read_csv(f'{transcript_dir}/{date}.csv', encoding='cp1252')
        data[date] = []
        v_length = video_lengths[date]

        for frame in frames[date]:
            start = frame['frame_index'] / 30 - window_radius
            end = frame['frame_index'] / 30 + window_radius
            start = max(start, 0.0)
            end = min(end, v_length)
            assert start < end
            transcript = transcripts[(transcripts['Global start'] < end) * (transcripts['Global end'] > start)]
            transcript = ' '.join(transcript['Sentence'].tolist())
            frame['video_start'] = start
            frame['video_end'] = end
            frame['transcript'] = transcript
            data[date].append(frame)
            
    os.makedirs('./context', exist_ok=True)
    with open(f'context/SR_w{window_length}s_{split}.json', 'w') as f:
        f.write(json.dumps(data, indent = 4))

In [4]:
# Construct context windows for the validation set with w = 60s
construct(split = 'val', window_length = 60)

Processing: 03152022
Processing: 11152022
